In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000

500000

In [7]:
function undrivenHelium!(τ, q, dq)
    tT, Q₁, Q₂, P₁, P₂ = q
    
    t1 = Q₁^2
    t2 = Q₂^2
    t = t1 * t2
    dq[1] = t
    R12 = t1 - t2
    R122 = R12^2
    RRR = (R122)^(3/2)
   
    dq[2] = 0.25*t2*P₁
    dq[4] = 2Q₁*(-0.125P₂^2 + Z*one(Q₁)  - t2 - t2/sqrt(R122) + t*R12/RRR)
    dq[3] = 0.25*t1*P₂
    dq[5] = 2Q₂*(-0.125P₁^2 + Z*one(Q₁)  - t1 - t1/sqrt(R122) - t*R12/RRR)

#    return [t, q₁, q₂, p₁, p₂] 
    nothing
end
#undrivenHelium(v) = undrivenHelium(zero(eltype(v)), v)


var1 = set_variables("t q1 q2 p1 p2", order = 1)


function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [0, Q1, Q2, P1, P2]
end

function regHam(t, Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(Q₁² - Q₂²)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

function errHam(N::Int)
    srand(487293456)
    J = [0 0 0 0 0; 0 0 0 1 0; 0 0 0 0 1; 0 -1 0 0 0; 0 0 -1 0 0 ]
    
    dnorm = zeros(N)
    als = typeof(zeros(5))[]
    for j in 1:N
        al = 10*rand(5)
        #al = [BigFloat(x) for x in al1]
        alt = al + var1
        ene = J*∇(regHam(alt))
        ene1 = [ene[k].coeffs[1].coeffs[1] for k in 1:5]
        meq = undrivenHelium(al)
        push!(als, al)
        meq[1] = 0
        dnorm[j] = norm(meq - ene1) 
    end
    return dnorm, als 
end

errHam (generic function with 1 method)

In [3]:
function q2sign(xold, x0, tol)
    if abs(xold[3]) < tol
        sq2o = 0
    else
        sq2o = sign(xold[3])
    end
    
    if abs(x0[3]) < tol
        sq2 = 0
    else
        sq2 = sign(x0[3])
    end
    if abs(xold[5]) < tol
        sp2o = 0
    else
        sp2o = sign(xold[5])
    end
    
    if abs(x0[5]) < tol
        sp2 = 0
    else
        sp2 = sign(x0[5])
    end
    return sq2o, sq2, sp2o, sp2
end
    

q2sign (generic function with 1 method)

In [4]:
a1 = condini(rand(2)...)
b1 = a1+var1
x = Array{Taylor1{TaylorN{Float64}}}(5)
for i in eachindex(b1)
    @inbounds x[i] = Taylor1( b1[i], order )
end

ord = x[1].order
xaux = Array{Taylor1{TaylorN{Float64}}}(5)
dx = Array{Taylor1{TaylorN{Float64}}}(5)
for j in eachindex(x)
    @inbounds xaux[j] = Taylor1( x[j].coeffs[1:ord] )
end

In [8]:
undrivenHelium!(0.0, xaux, dx)

LoadError: InterruptException:

In [6]:
xaux*xaux

LoadError: MethodError: no method matching *(::Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}, ::Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1})[0m
Closest candidates are:
  *(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  *([1m[31m::PyCall.PyObject[0m, ::Any) at /home/daniel/.julia/v0.5/PyCall/src/PyCall.jl:702
  *{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}([1m[31m::Union{Base.ReshapedArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:Union{Base.ReshapedArray{T,N,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{In